In [ ]:
#
# Copyright (C) Analytics Engines 2021
# Alastair McKinley (a.mckinley@analyticsengines.com)
#

import requests
import networkx as nx
from algorithmx import jupyter_canvas
from rich import print


r = requests.post("http://127.0.0.1:3001/rpc/login",json={"email" : "admin@beis.gov.uk", "password" : "Password1!"})
jwt = r.json()["signed_jwt"]

headers={
    "Content-Type" : "application/json",
    # "Accept": "application/vnd.pgrst.object+json",
    "Authorization": f"Bearer {jwt}"

}

params = {
    "select": "document_id:id,document_metadata_search:document_metadata_view(data,name,id),document_metadata:document_metadata_view(data,name),related_documents:document_graph!document_id_a(document_id_b,relationship_properties)",
    "document_metadata_search.id" : "eq.7",
    "document_metadata_search.data" : "wfts.Mersey Tunnels Act 2004",
    "document_metadata.name" : "in.(title,classification)"
}

r = requests.get("http://127.0.0.1:3001/document",headers=headers,params=params)

data = r.json()[0]
print(data)

G = nx.Graph()

canvas = jupyter_canvas().size((1200,1000))

name = [dm['data'] for dm in data["document_metadata"] if dm['name'] == 'title'][0]


for rd in data['related_documents']:
    nx.add_path(G,[name,rd['relationship_properties']['cited_in']])


canvas.nodes(G.nodes).add(
    shape='ellipse',
    size=(140,140)
)
canvas.edges(G.edges).add(
    size=100
)

canvas

